In [3]:
### Import Dependencies

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime


In [5]:
### Read data from crimecards.dc.gov, for the last year 2/1/22 - 2/5/23

In [6]:
# identify available columns
DC_crime = pd.read_csv('Resources/one_year_dc-crimes-search-results.csv')
DC_crime.columns

Index(['NEIGHBORHOOD_CLUSTER', 'offensegroup', 'CENSUS_TRACT', 'LONGITUDE',
       'END_DATE', 'offense-text', 'YBLOCK', 'DISTRICT', 'SHIFT', 'WARD',
       'YEAR', 'offensekey', 'BID', 'sector', 'PSA', 'ucr-rank', 'BLOCK_GROUP',
       'VOTING_PRECINCT', 'XBLOCK', 'BLOCK', 'START_DATE', 'CCN', 'OFFENSE',
       'OCTO_RECORD_ID', 'ANC', 'REPORT_DAT', 'METHOD', 'location',
       'LATITUDE'],
      dtype='object')

In [7]:
# convert data into a DataFram
DC_crime_data = pd.DataFrame(DC_crime)

# organize columns
DC_crime_data = DC_crime_data[['YEAR', 'START_DATE', 'offensegroup', 'OFFENSE', 'offense-text', 'offensekey']]

DC_crime_data.head()


,YEAR,START_DATE,offensegroup,OFFENSE,offense-text,offensekey
0,2022,"6/9/2017, 12:20:00 AM",violent,homicide,homicide,violent|homicide
1,2021,"2/11/2021, 3:00:52 PM",property,theft/other,theft/other,property|theft/other
2,2021,"2/11/2021, 6:03:44 PM",violent,robbery,robbery,violent|robbery
3,2021,"2/12/2021, 5:09:33 AM",violent,robbery,robbery,violent|robbery
4,2021,"2/18/2021, 4:39:52 AM",violent,robbery,robbery,violent|robbery


In [8]:
# slit the START_DATE into two columns, one for DATE and one for TIME
DC_crime_data[['START_DATE','START_TIME']] = DC_crime_data['START_DATE'].str.split(',',expand=True)

DC_crime_data.head()

,YEAR,START_DATE,offensegroup,OFFENSE,offense-text,offensekey,START_TIME
0,2022,6/9/2017,violent,homicide,homicide,violent|homicide,12:20:00 AM
1,2021,2/11/2021,property,theft/other,theft/other,property|theft/other,3:00:52 PM
2,2021,2/11/2021,violent,robbery,robbery,violent|robbery,6:03:44 PM
3,2021,2/12/2021,violent,robbery,robbery,violent|robbery,5:09:33 AM
4,2021,2/18/2021,violent,robbery,robbery,violent|robbery,4:39:52 AM


In [9]:
# drop rows with na values
DC_crime_data = DC_crime_data.dropna()

In [10]:
# convert 'START_TIME' to same format as found with weather data
DC_crime_data['START_TIME'] = pd.to_datetime(DC_crime_data['START_TIME'], format=' %I:%M:%S %p').dt.strftime('%H:%M:%S')

In [11]:
# convert 'START_DATE' to same format as found with weather data
DC_crime_data['START_DATE'] = pd.to_datetime(DC_crime_data['START_DATE'], format='%m/%d/%Y').dt.strftime('%Y/%m/%d')
DC_crime_data.head()

,YEAR,START_DATE,offensegroup,OFFENSE,offense-text,offensekey,START_TIME
0,2022,2017/06/09,violent,homicide,homicide,violent|homicide,00:20:00
1,2021,2021/02/11,property,theft/other,theft/other,property|theft/other,15:00:52
2,2021,2021/02/11,violent,robbery,robbery,violent|robbery,18:03:44
3,2021,2021/02/12,violent,robbery,robbery,violent|robbery,05:09:33
4,2021,2021/02/18,violent,robbery,robbery,violent|robbery,04:39:52


In [12]:
DC_crime_data.sort_values('START_DATE', ascending=True)

,YEAR,START_DATE,offensegroup,OFFENSE,offense-text,offensekey,START_TIME
42414,2022,1992/03/30,violent,homicide,homicide,violent|homicide,00:00:00
53261,2022,2004/09/30,violent,sex abuse,sex abuse,violent|sex abuse,05:30:00
13508,2021,2007/11/09,violent,sex abuse,sex abuse,violent|sex abuse,01:00:00
11706,2021,2008/07/01,violent,sex abuse,sex abuse,violent|sex abuse,20:00:31
35255,2021,2009/12/01,violent,sex abuse,sex abuse,violent|sex abuse,18:00:31
...,...,...,...,...,...,...,...
44829,2023,2023/02/07,property,theft/other,theft/other,property|theft/other,18:36:00
32868,2023,2023/02/07,violent,robbery,robbery,violent|robbery,22:15:00
32869,2023,2023/02/07,violent,robbery,robbery,violent|robbery,21:40:00
44990,2023,2023/02/07,property,motor vehicle theft,motor vehicle theft,property|motor vehicle theft,00:01:00


In [13]:
####Removed Start dates that happened prior to the 2/1/22 review period

In [20]:
# Convert the date to datetime64
DC_crime_data['START_DATE'] = pd.to_datetime(DC_crime_data['START_DATE'], format='%Y/%m/%d')
  
# Filter data between two dates
filtered_crime_data = DC_crime_data.loc[DC_crime_data['START_DATE'] >= '2022/02/06']
# Display

filtered_crime_data['START_DATE']


5017    2022-04-12
5034    2022-03-16
5035    2022-03-01
5036    2022-03-17
5037    2022-03-17
           ...    
56246   2023-02-01
56247   2023-02-05
56248   2023-02-06
56249   2023-02-07
56250   2023-02-04
Name: START_DATE, Length: 27697, dtype: datetime64[ns]

In [22]:
filtered_crime_data.groupby(DC_crime_data['offensekey']).count()
# filtered_crime_data.count()

,YEAR,START_DATE,offensegroup,OFFENSE,offense-text,offensekey,START_TIME
offensekey,,,,,,,
property|arson,8,8,8,8,8,8,8
property|burglary,1025,1025,1025,1025,1025,1025,1025
property|motor vehicle theft,4094,4094,4094,4094,4094,4094,4094
property|theft f/auto,7908,7908,7908,7908,7908,7908,7908
property|theft/other,10910,10910,10910,10910,10910,10910,10910
violent|assault w/dangerous weapon,1390,1390,1390,1390,1390,1390,1390
violent|homicide,198,198,198,198,198,198,198
violent|robbery,2008,2008,2008,2008,2008,2008,2008
violent|sex abuse,156,156,156,156,156,156,156


In [22]:
#### Export clean crime data to new cvs file

In [26]:
filtered_crime_data.to_csv("Resources/clean_crime_data.csv")

In [27]:
DC_crime_clean = pd.read_csv("Resources/clean_crime_data.csv")
DC_crime_clean.head()

,Unnamed: 0,YEAR,START_DATE,offensegroup,OFFENSE,offense-text,offensekey,START_TIME
0,5017,2022,2022-04-12,violent,robbery,robbery,violent|robbery,20:00:00
1,5034,2022,2022-03-16,property,theft f/auto,theft f/auto,property|theft f/auto,17:00:00
2,5035,2022,2022-03-01,property,theft f/auto,theft f/auto,property|theft f/auto,16:00:00
3,5036,2022,2022-03-17,property,motor vehicle theft,motor vehicle theft,property|motor vehicle theft,10:45:00
4,5037,2022,2022-03-17,property,motor vehicle theft,motor vehicle theft,property|motor vehicle theft,16:45:00


In [28]:
DC_crime_clean = pd.DataFrame(DC_crime_clean)


In [29]:
total_days = DC_crime_clean['START_DATE'].value_counts()
total_days

2022-08-14    106
2022-11-25    106
2022-07-30    105
2023-01-28    102
2022-11-12    101
             ... 
2022-04-14     52
2022-11-24     50
2022-04-03     49
2022-03-12     49
2022-05-02     47
Name: START_DATE, Length: 367, dtype: int64